In [ ]:
import logging
import pickle
from itertools import combinations
from pathlib import Path

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, PolynomialFeatures
from sklearn.svm import SVC

In [ ]:
logging.basicConfig(level=logging.INFO)

data_dir = Path("../data")
y_path = data_dir / "y.pkl"
features_path = data_dir / "tsfresh_features_df.pkl"

In [ ]:
with open(features_path, "rb") as f:
    X = pickle.load(f)

with open(y_path, "rb") as f:
    y = pickle.load(f)

In [ ]:
clf = Pipeline([
    ("features", PolynomialFeatures(degree=2)),
    ("scale", QuantileTransformer(n_quantiles=20)),
    ("model", SVC(kernel="poly", degree=3, class_weight={0:1.5, 1:1}))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
grid = GridSearchCV(
    estimator=clf,
    param_grid={
        "model__C": [1],
        "model__kernel": ["poly"],
        "model__degree": [3],
        "model__gamma": ["scale"],
        "model__class_weight": [{0: weight, 1: 1} for weight in [1.5]]
    },
    cv=10,
    n_jobs=-1,
    verbose=1,
)
grid.fit(X, y)

In [ ]:
pd.DataFrame(grid.cv_results_)